In [172]:
import sys
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER/')
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint
from numpy import newaxis
import sklearn
import subprocess
import fastText

In [38]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]


def createMatrices(sentences, label2Idx, case2Idx,char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}

        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []
        caseIndices = []
        charIndices = []
        labelIndices = []
        
        for word,char,label in sentence:  
            charIdx = []
            for x in char:
                if x in char2Idx.keys():
                    charIdx.append(char2Idx[x])
                else:
                    charIdx.append(char2Idx['UNKNOWN'])
            #Get the label and map to int            
            wordIndices.append(word)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, charIndices, labelIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences



In [39]:
epochs = 80
trainable = True

In [40]:
from keras_contrib.layers import CRF


In [41]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [ ]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [42]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [ ]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [43]:
trainSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


23999
2199
5099


In [ ]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


In [44]:
print(trainSentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [45]:
labelSet = set()
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)

In [46]:
print(len(labelSet))

25


In [47]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [48]:
print(label2Idx)

{'I-LOCderiv': 0, 'B-LOC': 1, 'B-PERpart': 14, 'I-LOCpart': 4, 'B-OTH': 15, 'I-ORGderiv': 17, 'B-ORGderiv': 5, 'B-LOCpart': 19, 'O': 6, 'I-LOC': 7, 'I-PERderiv': 16, 'B-OTHderiv': 2, 'I-OTHderiv': 8, 'I-ORGpart': 21, 'I-PER': 3, 'B-PER': 22, 'B-OTHpart': 11, 'B-PERderiv': 9, 'I-PERpart': 24, 'B-ORG': 10, 'B-LOCderiv': 23, 'I-OTH': 18, 'I-ORG': 12, 'B-ORGpart': 13, 'I-OTHpart': 20}


In [13]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [14]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 4, 'allLower': 1, 'numeric': 0, 'mainly_numeric': 5, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'allUpper': 2, 'initialUpper': 3}


In [15]:
print(type(trainSentences))

<class 'list'>


In [49]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [50]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'Ö': 138, 'к': 1, 'Ş': 2, 'Á': 176, '⊃': 3, 'č': 4, '€': 232, '\x9a': 5, 'Ł': 6, ')': 7, '▪': 8, '°': 9, '·': 56, 'UNKNOWN': 328, '太': 13, 'Ž': 12, '傳': 188, '2': 180, 'Z': 242, '妃': 252, '§': 11, 'ā': 14, 'ó': 15, '~': 297, '동': 17, 'ź': 82, 'İ': 18, 'I': 19, 'т': 25, '.': 21, '“': 23, 'ế': 0, 'Π': 31, '別': 61, 'Т': 33, '@': 27, 'ǒ': 281, 'л': 30, 'ę': 32, '*': 275, 'p': 291, 'Â': 42, '_': 38, 'н': 36, 'ь': 37, 'ú': 308, 'r': 39, 'R': 40, ',': 41, 's': 44, '柯': 45, '+': 46, 'Ä': 47, 'α': 48, 'è': 49, 'k': 223, 'È': 68, 'œ': 52, 'c': 53, 'г': 54, 'ъ': 57, '$': 58, ';': 59, 'e': 60, 'B': 62, 'A': 63, '\x95': 64, 'é': 66, 'o': 65, 'и': 67, '≤': 312, '\x80': 313, 'ʻ': 69, 'λ': 70, 'ņ': 71, 'y': 224, 'û': 73, 'æ': 75, '⋅': 76, 'ö': 77, '守': 79, '貴': 81, 'κ': 55, '—': 83, '李': 84, '©': 86, 'ã': 131, 'Å': 87, 'ρ': 88, '−': 89, '<': 90, '×': 243, 'б': 91, 'η': 239, 'ς': 93, 'р': 94, 'h': 95, 'Q': 96, 'î': 97, '’': 10, 'τ': 99, 'x': 133, 'ж': 100, '\x96': 101, 'о': 102, 'с': 270, 'ō': 105, 'š

In [18]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [51]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [54]:
print(len(trainSentences[0]))

25


In [55]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [66]:
def createBatches(dataset):
    l = []
    for i in dataset:
        l.append(len(i))
    l = set(l)
    print(len(l))
    batches = []
    batch_len = []
    z = 0
    for i in l:
        temp = []
        for batch in dataset:
            if len(batch) == i:
                temp.append(batch)
                z += 1
        batches.append(temp)
#         batch_len.append(z)
    return batches

In [82]:
train_batches = createBatches(trainSentences)
dev_batches = createBatches(devSentences)


51
45


In [72]:
print(batches[0][0])
print(batches[0][1])
print(len(batches))
print(len(batches[0]))
print(batches[1][0])

[['Alles', 'O'], ['richtig', 'O'], ['.', 'O']]
[['Farben', 'O'], ['eingeführt', 'O'], ['.', 'O']]
51
3
[['Brennt', 'O'], ['nichts', 'O'], ['nieder', 'O'], ['.', 'O']]


In [87]:
def generator(batches: 'list of training/dev sentences- batches already created'):
    global line_number
    
    while True:
        total_lines = len(dataset)
        for batch in batches:
            word_embeddings = []
            case_embeddings = []
            char_embeddings = []

            output_labels = []
            for index in range(len(batch)): # batches made according to the size of the sentences. len(batch) gives the size of current batch            
#                 index = line_number%total_lines
#                 line_number+=1
                sentence = batch[index]
    #             print(sentence)
                temp_casing = []
                temp_char=[]
                temp_word=[]
                temp_output=[]
                for word in sentence:
                    word, label = word
                    casing =getCasing(word, case2Idx)
                    temp_casing.append(casing)
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)
                    word_vector = ft.get_word_vector(word.lower())
                    temp_word.append(word_vector)
                    temp_output.append(label2Idx[label])
                temp_char = pad_sequences(temp_char, 52)
                word_embeddings.append(temp_word)
                case_embeddings.append(temp_casing)
                char_embeddings.append(temp_char)
                temp_output = to_categorical(temp_output, 25)
                output_labels.append(temp_output)
    #             output_labels = to_categorical()
    #             output_labels = np.array(output_labels)
    #             output_labels = output_labels[...,newaxis]

    #             print(np.array(word_embeddings).shape)
    #             print(np.array(case_embeddings).shape)
    #             print(np.array(char_embeddings).shape)
    #             print(output_labels.shape)
    #             print("******************\n\n")
            yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [78]:
line_number=0
for inp, out in train_generator(batches):
    word, case, char = inp
    print(word.shape)
    print(case.shape)
    print(char.shape)
    print(out.shape)
    print("********")

(3, 3, 300)
(3, 3)
(3, 3, 52)
(3, 3, 25)
********
(32, 4, 300)
(32, 4)
(32, 4, 52)
(32, 4, 25)
********
(127, 5, 300)
(127, 5)
(127, 5, 52)
(127, 5, 25)
********
(317, 6, 300)
(317, 6)
(317, 6, 52)
(317, 6, 25)
********
(497, 7, 300)
(497, 7)
(497, 7, 52)
(497, 7, 25)
********
(755, 8, 300)
(755, 8)
(755, 8, 52)
(755, 8, 25)
********
(913, 9, 300)
(913, 9)
(913, 9, 52)
(913, 9, 25)
********


KeyboardInterrupt: 

In [25]:
idx2Label = {v: k for k, v in label2Idx.items()}


In [79]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [194]:
words_input = Input(shape=(None,300),dtype='float32',name='words_input')
# words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
# model.compile(loss='categorical_crossentropy', optimizer='nadam', )
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_67 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_70 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [112]:
checkpoint = ModelCheckpoint('generator_NER_best.h5', monitor='f1_metric', verbose=1, save_best_only=True, period=1)


In [ ]:
# line_number=0
model.reset_states()
max_f1=0
for i in range(80):
    print("epoch %d" %i)
    model.fit_generator(generator(train_batches), epochs=1, steps_per_epoch= len(batches))
    
    print(len(devSentences))
    
    if i>=50:
        pred_labels=[]
        correct_labels=[]
        for sentence in devSentences:
            pred_labels_sentence=[]
            correct_labels_sentence=[]
            temp_sentence=[]
            for word in sentence:
                w, correct_label = word
                correct_labels_sentence.append(label2Idx[correct_label])
                temp_sentence.append(w)
            pred_label = get_predictions(temp_sentence, model)
            pred_label = np.ndarray.tolist(pred_label[0])
        #     print(len(pred_label[0]))
            for label in pred_label:
                label = label.index(max(label))
                pred_labels_sentence.append(label)
            pred_labels.append(pred_labels_sentence)
            correct_labels.append(correct_labels_sentence)
        pre_dev, rec_dev, f1_dev = compute_f1(pred_labels, correct_labels, idx2Label)
        print(f1_dev)
        if max_f1 < f1_dev:
            print("model saved")
            model.save('generator_NER_best.h5')
            max_f1 = f1_dev
    #     print((len(correct_labels), len(pred_labels)))
#     print(f1_score(correct_labels, pred_labels, average='macro'))
    
    
    

epoch 0
Epoch 1/1
21/51 [===========>..................] - ETA: 12s - loss: -0.5313 - acc: 0.9874

In [190]:
pred_labels=[]
correct_labels=[]
print(len(devSentences))
for sentence in devSentences:
    pred_labels_sentence=[]
    correct_labels_sentence=[]
    temp_sentence=[]
    for word in sentence:
        w, correct_label = word
        correct_labels_sentence.append(label2Idx[correct_label])
        temp_sentence.append(w)
    pred_label = get_predictions(temp_sentence, model)
    pred_label = np.ndarray.tolist(pred_label[0])
#     print(len(pred_label[0]))
    for label in pred_label:
        label = label.index(max(label))
        pred_labels_sentence.append(label)
    pred_labels.append(pred_labels_sentence)
    correct_labels.append(correct_labels_sentence)
print((len(pred_labels), len(correct_labels)))

2199
(2199, 2199)


In [191]:
pre_dev, rec_dev, f1_dev = compute_f1(pred_labels, correct_labels, idx2Label)

In [192]:
print(f1_dev)

0.8189239332096476


In [189]:
model.load_weights('generator_NER_best.h5')

In [178]:
for sentence in devSentences:
    temp_sentence=[]
    for word in sentence:
        w, correct_label = word
        correct_labels.append(label2Idx[correct_label])
        temp_sentence.append(w)
    pred_label = get_predictions(temp_sentence, model)
    pred_label = np.ndarray.tolist(pred_label[0])
#     print(len(pred_label[0]))
    for label in pred_label:
        label = label.index(max(label))
        pred_labels.append(label)
temp_f1 = f1_score(correct_labels, pred_labels, average='macro')
        

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [183]:
temp_f1 = f1_score(correct_labels, pred_labels, average='macro')
print(temp_f1)

0.5227800135258113


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [150]:
def convert_sentence_to_input(sentence):
    word_embeddings = []
    case_embeddings = []
    char_embeddings = []

#     output_labels = []
    temp_casing = []
    temp_char=[]
    temp_word=[]
#     temp_output=[]
    for word in sentence:
        casing =getCasing(word, case2Idx)
        temp_casing.append(casing)
        temp_char2=[]
        for char in word:
            if char in char2Idx.keys():
                temp_char2.append(char2Idx[char])
            else:
                temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
        temp_char2 = np.array(temp_char2)
        temp_char.append(temp_char2)
        word_vector = ft.get_word_vector(word.lower())
        temp_word.append(word_vector)
#         temp_output.append(label2Idx[label])
    temp_char = pad_sequences(temp_char, 52)
    word_embeddings.append(temp_word)
    case_embeddings.append(temp_casing)
    char_embeddings.append(temp_char)
#     temp_output = to_categorical(temp_output, 25)
#     output_labels.append(temp_output)
    ans = [np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)]
    return ans



In [152]:
def get_predictions(sentence, model):
    inp = convert_sentence_to_input(sentence)
    
    out = model.predict(inp)
    return out

In [151]:
for sentence in devSentences:
        for word in sentence:
            w, label = word
            temp_sentence = []
            temp_sentence.append(w)
        x = convert_sentence_to_input(temp_sentence)
#         print(pred_label)